In [1]:
import pandas as pd
from sklearn import preprocessing
from pyod.models.knn import KNN

In [87]:
data = pd.read_csv("../outputs/clean_houses.csv", index_col=0)

In [88]:
data.head()

,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_pool,buy_price,buy_price_by_area,latitude,longitude
0,"San Cristóbal, Villaverde(Madrid)",HouseType 1: Pisos,False,False,64.0,2,1.0,3,True,False,False,False,False,False,False,85000,1328,40.343263,-3.688352
1,"Los Ángeles, Villaverde(Madrid)",HouseType 1: Pisos,True,False,70.0,3,1.0,4,True,True,False,False,True,False,False,129900,1856,40.355109,-3.700105
2,"San Andrés, Villaverde(Madrid)",HouseType 1: Pisos,False,False,94.0,2,2.0,1,True,True,False,True,False,False,False,144247,1535,40.345539,-3.710970
3,"San Andrés, Villaverde(Madrid)",HouseType 1: Pisos,False,False,64.0,2,1.0,Bajo,True,True,False,True,False,False,False,109900,1717,40.345539,-3.710970
4,"Los Rosales, Villaverde(Madrid)",HouseType 1: Pisos,False,False,108.0,2,2.0,4,True,True,True,True,False,False,True,260000,2407,40.355841,-3.688399


In [89]:
def transf(column):

    le = preprocessing.LabelEncoder()
    data[column] = le.fit_transform(data[column])
    return data[column]

In [90]:
transf("house_type_id")
transf("is_renewal_needed")
transf("is_new_development")
transf("is_exterior")
transf("has_lift")
transf("has_parking")
transf("has_storage_room")
transf("has_terrace")
transf("has_balcony")
transf("has_pool");

In [91]:
planta={"Bajo": 0,
       "Entreplanta exterior": -1,
        "Entreplanta interior": -1,
       "Semi-sótano exterior": -2,    
        "Semi-sótano interior": -2,   
        "Sótano interior": -3,          
        "Sótano exterior": -3}

In [92]:
data.floor = data.floor.apply(lambda x: planta.get(x, x))
data.floor = pd.to_numeric(data.floor)

In [93]:
"""data = data.drop(data[data["buy_price"]>1000000].index)
data = data.drop(data[data["sq_mt_built"]>200].index)
data = data.drop(data[data["n_rooms"]>3].index)
data = data.drop(data[data["n_bathrooms"]>3].index)
data = data.drop(data[data["floor"]<0].index)"""

'data = data.drop(data[data["buy_price"]>1000000].index)\ndata = data.drop(data[data["sq_mt_built"]>200].index)\ndata = data.drop(data[data["n_rooms"]>3].index)\ndata = data.drop(data[data["n_bathrooms"]>3].index)\ndata = data.drop(data[data["floor"]<0].index)'

In [94]:
clf = KNN(contamination=0.5, method="largest", algorithm="auto")
clf.fit(data)
y_pred = clf.predict(data)
dropea = data[y_pred == 1]
data = data.drop(dropea.index)

In [95]:
clf = KNN(contamination=0.5, method="largest", algorithm="auto")
clf.fit(data)
y_pred = clf.predict(data)
dropea = data[y_pred == 1]
data = data.drop(dropea.index)

In [96]:
data.reset_index()
data

,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_pool,buy_price,buy_price_by_area,latitude,longitude
1,67,0,1,0,70.0,3,1.0,4,1,1,0,0,1,0,0,129900,1856,40.355109,-3.700105
4,66,0,0,0,108.0,2,2.0,4,1,1,1,1,0,0,1,260000,2407,40.355841,-3.688399
5,100,0,0,0,126.0,4,2.0,3,1,0,1,0,1,1,0,195000,1548,40.345539,-3.710970
6,100,0,0,0,120.0,5,2.0,1,1,0,1,1,1,1,1,190000,1583,40.345539,-3.710970
10,66,0,0,0,123.0,3,2.0,4,1,1,0,0,0,0,0,195000,1585,40.355841,-3.688399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17377,63,0,0,0,113.0,2,1.0,1,1,1,1,1,0,0,1,425000,3761,40.391172,-3.695190
17381,63,0,0,0,228.0,4,2.0,5,1,1,1,1,1,1,0,895000,3925,40.391172,-3.695190
17384,110,0,0,0,117.0,3,2.0,1,1,1,1,0,0,0,0,349000,2983,40.473171,-3.584152
17393,21,0,0,0,88.0,2,2.0,1,1,1,1,1,0,0,1,360000,4420,40.459749,-3.612738


In [97]:
# Aplicamos ahora get_dummies a la columna "house_type_id" y "floor".

# data = pd.get_dummies(data, columns = ["house_type_id"])

In [98]:
data.columns

Index(['neighborhood_id', 'house_type_id', 'is_renewal_needed',
       'is_new_development', 'sq_mt_built', 'n_rooms', 'n_bathrooms', 'floor',
       'is_exterior', 'has_lift', 'has_parking', 'has_storage_room',
       'has_terrace', 'has_balcony', 'has_pool', 'buy_price',
       'buy_price_by_area', 'latitude', 'longitude'],
      dtype='object')

In [99]:
data = data[['neighborhood_id', 'house_type_id', 'is_renewal_needed',
       'is_new_development', 'sq_mt_built', 'n_rooms', 'n_bathrooms', 'floor',
       'is_exterior', 'has_lift', 'has_parking', 'has_storage_room',
       'has_terrace', 'has_balcony', 'has_pool', 'buy_price']]

### Iniciamos el análisis con H2O

In [100]:
import h2o
from h2o.estimators import H2ORandomForestEstimator

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 33 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_Javi_cefkpb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,742 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [101]:
h2train = h2o.H2OFrame(data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [102]:
columnas = [col for col in data.columns if col not in ["buy_price"]]

x = list(columnas)

y = "buy_price"

In [103]:
from h2o.automl import H2OAutoML

automl = H2OAutoML(max_runtime_secs=1200, sort_metric='RMSE')
automl.train(x, y, training_frame=h2train)
leader_board = automl.leaderboard
leader_board.head()

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201013_195652,72523.1,5.2596e+09,5.2596e+09,41398.7,0.163042
StackedEnsemble_BestOfFamily_AutoML_20201013_195652,75480,5.69724e+09,5.69724e+09,45435.7,0.175136
XGBoost_grid__1_AutoML_20201013_195652_model_4,79135.1,6.26236e+09,6.26236e+09,46524.2,0.183197
XGBoost_grid__1_AutoML_20201013_195652_model_7,80498.9,6.48008e+09,6.48008e+09,46833.6,0.177987
XGBoost_grid__1_AutoML_20201013_195652_model_5,80742.6,6.51937e+09,6.51937e+09,44320.1,0.179022
GBM_1_AutoML_20201013_195652,80979.5,6.55768e+09,6.55768e+09,49848.6,0.186661
XGBoost_grid__1_AutoML_20201013_195652_model_8,81512.6,6.6443e+09,6.6443e+09,45124,0.193344
XGBoost_3_AutoML_20201013_195652,81595.3,6.65779e+09,6.65779e+09,48412.6,0.178819
GBM_grid__1_AutoML_20201013_195652_model_9,81741.8,6.68173e+09,6.68173e+09,47523.7,0.184165
XGBoost_grid__1_AutoML_20201013_195652_model_11,82105.9,6.74139e+09,6.74139e+09,50803,0.183972


In [112]:
import numpy as np

In [117]:
dato = []
for i in range (10):
    dato.append (+1)
print(np.sum(dato)/10)

1.0
